In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from re import sub
from sympy import sympify, Symbol, exp, log, I, pi, zoo, latex, N, simplify
from scipy.optimize import curve_fit 

In [2]:
def safe_average(values, index):
    # This function computes the average of numeric values at a specified index in a list of lists, while safely filtering out entries that are not lists, too short, or have non-numeric values at that index.

    selected = [
        item[index] for item in values
        if isinstance(item, list)
        and len(item) > index
        and isinstance(item[index], (int, float))
    ]
    return sum(selected) / len(selected) if selected else float('nan')

In [3]:
def parse_model_string(model_str):
    # It turns what was stored in the text file as a string back to a list with a sympy object for the expression obtained during the training.

    # Locals for SymPy expressions
    converter = {
        'exp': exp,
        'log': log,
        'I': I,
        'pi': pi
    }

    try:
        # Replace np.float64(...) with just the number
        model_str_clean = sub(r'np\.float64\((.*?)\)', r'\1', model_str)
        
        # Find the last comma (before the expression) and quote the expression part
        if model_str_clean.strip().startswith('[') and model_str_clean.strip().endswith(']'):
            inside = model_str_clean[1:-1]  # Remove surrounding []
            parts = inside.rsplit(',', 1)  # Split on last comma
            expression_quoted = f'"{parts[1].strip()}"'
            model_str_safe = f'[{parts[0].strip()}, {expression_quoted}]'
        else:
            raise ValueError("Malformed model string")

        # Safely evaluate the list
        parts = eval(model_str_safe, {"__builtins__": None}, {})

        # Convert the last part (string) to a SymPy expression
        parts[-1] = sympify(parts[-1], locals=converter)

        return parts

    except Exception as e:
        print(f"Error parsing model:\n{model_str}\n{e}\n")
        return None

In [4]:
def process(data):
    # Solves an occuring issue in visualize_set.
    return data  # no copy, just returns the original reference

In [5]:
def visualize_set(data_set, ax, estimator = False, add_legend=True):
    # This allows to see how the data is spread. The raw data can be seen for any of the X_train, X_test or the total data X set. If a gplearn or a lamdified object is given the predicted data is visualized.

    # The data must be reloaded each time as a clean look was wanted for the symbolic_regressor.ipynb file. It slows the process but makes the other file clearer. 
    Energies_train = pd.read_csv('Fe_Octhedral_complexes_rel_m_splitting.csv')
    Energies_test = pd.read_csv('real_world_energies_and_relative_metal_radius.csv')
    Energies_total = pd.concat([Energies_train,  Energies_test], ignore_index=True, axis=0)

    X_train = Energies_train[['rel_m']]
    X_test = Energies_test[['rel_m']]
    X = Energies_total[['rel_m']]
    

    number_of_plots = 4 # (Multiplicities 1,2, 5 and 6)

    # When inputting an estimator in the function the name of the plot changes from the Reference data (data points calculated using DFT) to Estimated data (data points estimated with the expression obtained)
    if not estimator:
        ax.set_prop_cycle('color', plt.cm.cool(np.linspace(0, 1, number_of_plots)))
        title = 'Reference data'
    else:
        ax.set_prop_cycle('color', plt.cm.hot(np.linspace(0, 0.9, number_of_plots)))
        title = 'Estimated data'

    # Labels x and y axes.
    ax.set(xlabel = 'Spin splitting energy [kcal/mol]', ylabel = r'$\mathrm{r_{rel}}$')

    i = 0

    # Identifies which set must be considered
    if data_set.equals(X_train):
        Energies = Energies_train
    elif data_set.equals(X_test):
        Energies = Energies_test
    elif data_set.equals(X):
        Energies = Energies_total


    # Plots the data points for one multiplicity at a time
    for Multiplicity in [1,2,5,6]:
        splitting_E = []
        r_rel = []

        # Loops over the complexes in the data set considered.
        for idx in range(len(Energies)):     
            
            # If the multiplicity currently looked at corresponds to the multiplicity of the complex, the data point is added to the list of data points to scatter in this current loop.
            if Multiplicity == Energies.loc[idx,'multiplicity']:
                r_rel.append(Energies.at[idx, 'rel_m'])
                
                # If Reference data: takes the splitting energy in the dataset.
                if not estimator:
                    splitting_E.append(Energies.at[idx, 'splitting'])

                # Else calculates it using the expression given. Two cases: the estimator is a numpy object (first case) or it is a gplearn object (second case).
                else:
                    if type(estimator).__name__ == "function":
                        splitting_E.append(estimator(Energies.at[idx, 'rel_m']))
                    else:
                        splitting_E.append(estimator.predict([[Energies.at[idx, 'rel_m']]]))

        if not estimator:
            label = f'{Multiplicity} (data)'
        else:
            label = f'{Multiplicity} (estimated)'

        # After the loop over all the complexes the points corresponding to the multiplicity are scattered on the plot with the color given by the prop_cycle.
        ax.scatter(splitting_E, r_rel, edgecolors='k', linewidth=0.5, label=label)
            
                

            
    if data_set.equals(X_train):
        title += ': training set'
    elif data_set.equals(X_test):
        title += ': testing set'
    elif data_set.equals(X):
        title += ': total set'
        
    if add_legend == True:
        ax.set_title(title)
        plt.legend()
    plt.grid(False)
    # Add a subtle background color
    ax.set_facecolor('#f9f9f9')
    
    # Remove spines for cleaner appearance
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    plt.savefig('Images/Training_set.svg', bbox_inches='tight')

    return



In [6]:
def visualize_estimator(estimator):
    # Allows to visualize how the estimator performs against the test set. Its expression is plotted at the top and a parity plot is also made. 

    # The data must be reloaded each time as a clean look was wanted for the symbolic_regressor.ipynb file. However, it makes the process slower. 
    Energies_test = pd.read_csv('real_world_energies_and_relative_metal_radius.csv')
    X_test = Energies_test[['rel_m']]
    y_test = Energies_test['splitting']
    
    # Locals for SymPy expressions
    converter = {
        'sub': lambda x, y : x - y,
        'div': lambda x, y : x/y,
        'mul': lambda x, y : x*y,
        'add': lambda x, y : x + y,
        'neg': lambda x    : -x,
        'pow3': lambda x : x**3,
        'root3': lambda x : x**(1/3)
    }

    
    # The plots are created and the labels are added.
    fig, axs = plt.subplots(ncols=2, figsize=(10, 6), dpi = 150)  # Two horizontal subplot
    (ax1, ax2) = axs
    
    ax1.set_xlabel('Spin splitting energy [kcal/mol]', fontsize= 12)
    ax1.set_ylabel(r'$\mathrm{r_{rel}}$', fontsize= 12)
    ax1.set_title('Visualization of predictor', fontsize= 14)

    ax2.set_xlabel('DFT calculated spin splitting energy [kcal/mol]', fontsize= 12)
    ax2.set_ylabel('Predicted spin splitting energy [kcal/mol]', fontsize=12)
    ax2.set_title('Accuracy of fit', fontsize= 14)
    plt.subplots_adjust(right=0.95)

    
    # The predicted data and the testing data are put on the first axis without their legen being added.
    visualize_set(X_test, ax1, estimator = False, add_legend = False)
    visualize_set(X_test, ax1, estimator = estimator, add_legend = False)

    # The analytical expression is plotted on the first axis.
    y_see_function = pd.DataFrame(np.arange(min(X_test['rel_m']), max(X_test['rel_m']), 0.001))
    
    if type(estimator).__name__ == "function":
        x_see_function = [estimator(y_see_function[0][i]) for i in range(len(y_see_function))]
    else:
        x_see_function = estimator.predict(y_see_function)
    
    ax1.plot(x_see_function, y_see_function, linestyle='-', color = 'k', markersize=3, zorder=2)

    # The parity plot is created here by recreating the predicted values from the estimator. It can both be a gplearn object or a numpy expression.
    if type(estimator).__name__ == "function":
        y_pred = [estimator(X_test['rel_m'][i]) for i in range(len(X_test['rel_m']))]
    else:
        y_pred = estimator.predict(X_test)

    ax2.scatter(list(y_test), y_pred, color = 'b', edgecolors='k', linewidth=0.5)

    # The information on how well the model performed are calculated here.
    popt_lin_reg, pcov_lin_reg = curve_fit(linear_regression, list(y_test), list(y_pred), bounds = (0, np.inf))
    y_fit_lin_reg = linear_regression(y_test, *popt_lin_reg)
    r_squared_lin_reg = r_squared_function(y_fit_lin_reg, y_pred)
    MAE = estimator._program.raw_fitness_

    # The linear regression curve is drawn on the second plot for only two points for efficiency purposes.
    plot_limits =  [1.15*min(min(y_test), min(y_pred)), 1.15*max(max(y_test), max(y_pred))]
    y_fit_lin_reg_plot = linear_regression(plot_limits, *popt_lin_reg)
    ax2.plot(plot_limits, y_fit_lin_reg_plot, color = 'k', linewidth = 0.5, label = r'linear regression: $\mathrm{R^2 = %.3f}$' '\n'
                                                                            r'linear regression: $\mathrm{slope = %.3f}$' '\n' 
                                                                            r' MAE = %.3f'% (r_squared_lin_reg, popt_lin_reg[0], MAE))

    # Legends are added and the secon plot has its axes set to a certain size to show best the slope of the estimated data.
    ax2.set_xlim(plot_limits)
    ax2.set_ylim(plot_limits)
    ax1.legend(title = 'multiplicity', ncol=2, fontsize = 10)
    ax2.legend(fontsize = 10, loc = 'upper left')

    
    # The expression is added at the top of the plot.
    expr_str = sympify(estimator._program.__str__(), locals = converter)
    # Defines the replacement symbol
    X = Symbol('X0') 
    r_rel = Symbol('r_{rel}')
    rounded_expr = expr_str.xreplace({
        **{n: round(N(n), 3) for n in expr_str.atoms() if n.is_Float},
        X: r_rel
    })
    latex_expr = r"\Delta E = " + latex(rounded_expr)
    fig.suptitle(f"${latex_expr}$", y = 1.0, fontsize= 20)

    
    # Makes the plot look neat by setting a beackground color and removing the top and right black lines of the plot.
    ax2.set_facecolor('#f9f9f9')
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)

    # Saves the image according to the size of the plot accounting for the suptitle.
    plt.tight_layout(rect=[0, 0, 1, 1]) 
    plt.savefig("Images/First_good_model.png", dpi=300, bbox_inches='tight')

    return MAE, r_squared_lin_reg, popt_lin_reg[0]


In [7]:
def get_MAE_r_squared_slope_of_est(estimator):
    # The name of the function is self explanatory, from an estimator you obtain the MAE, the r squared and the slope of the parity plot compared to the testing data.
    # This function only works for gplearn estimator objects. This function can be used in the obtention of the different models as to have a faster calculation process.
    
    # The data must be reloaded each time as a clean look was wanted for the symbolic_regressor.ipynb file. It slows the process but makes the other file clearer. 
    Energies_test = pd.read_csv('real_world_energies_and_relative_metal_radius.csv')
    X_test = Energies_test[['rel_m']]
    y_test = Energies_test['splitting']

    # The predicted data is obtained from the expression of the model and the testing set. 
    y_pred = estimator.predict(X_test)

    # The characteristics are calculated
    popt_lin_reg, pcov_lin_reg = curve_fit(linear_regression, list(y_test), list(y_pred), bounds = (0, np.inf))
    y_fit_lin_reg = linear_regression(y_test, *popt_lin_reg)
    r_squared_lin_reg = r_squared_function(y_fit_lin_reg, y_pred)
    MAE = estimator._program.raw_fitness_

    return MAE, r_squared_lin_reg, popt_lin_reg[0]

In [8]:
def visualize_models(df, type_tuning):
    """
    This function creates pie charts of how well the trained model performed. The models are "ranked" based on if their expression contained a singularity in the range of training/testing data, 
    if the r squared is positive, if the expression contained unphysical terms (infinite or imaginary values) an if the ground spin state prediction was well done.
    
    The dataframe given has this structure: its columns are the name of the models (I, II; III, IV, V and VI) and the rows are lists which are in this order: 
    singularity_match, r_squared, percentage_same_sign(y_test_sorted, y_pred), MAE, slope, expr_str
    """
    
    for key in list(df.keys()):
        df[key] = df[key].apply(parse_model_string)

    data_list = []
    good_models = []

    # Goes through all the models of a certain type of training and counts how many models fill some criteria.
    for model_type in df.keys():
        data = [0, 0, 0, 0, 0, 0]
        
        for idx_model, model in enumerate(df[model_type]):
            if model[0]:
                data[0]+=1
                continue
                
            if model[1] < 0:
                data[1]+=1
                continue

            if model[5].has(I) or model[5].has(zoo):
                data[2]+=1
                continue
                
            if model[2] < 90:
                data[3]+=1
                continue

            if model[2] < 98:
                data[4]+=1
                continue

            # The models that don't correspond to any of those criteria are added to the category of good models.
            data[5]+=1
            good_models.append([model_type, idx_model, model])

        data_list.append(np.array(data)/np.sum(data)*100)

    # Defines the title of the pie charts and the labels to be put on the right side of the plot. 
    titles = ['I', 'II', 'III', 'IV', 'V', 'VI']
    global_labels = ['Singularity', r'negative $\mathrm{R^2}$', 'Imaginary part/infinite term', 'Spin agreement < 90%', '90% ≤ Spin agreement < 98%', 'Good models']
    colors = plt.cm.cool(np.linspace(0, 1, len(global_labels)))
    
    # Create 2x3 subplots and adds the pie charts to each.
    fig, axes = plt.subplots(2, 3, figsize=(8, 6))
    axes = axes.flatten()  # Flatten to make indexing easier
    
    for i in range(6):
        wedges, _ = axes[i].pie(data_list[i], colors=colors)
        axes[i].set_title(f"{titles[i]}: MAE = {round(safe_average(df[titles[i]], 3), 2)}")
    
    # Global legend (top right of figure)
    fig.legend(wedges, global_labels, loc='upper right', bbox_to_anchor=(1.3, 1), title='Categories')

    # Changes the title according to the two type of models trained. It has to be entered by hand which experience it was.
    if type_tuning == 'Add' or type_tuning == 'add':
        suptitle = "Distribution of models based on the tuned\n parameters on Add MAE"
    if type_tuning == 'Exp' or type_tuning == 'exp':
        suptitle = "Distribution of models based on the tuned\n parameters on Exp spike/cst"

    fig.suptitle(suptitle, fontsize=18)
    
    plt.tight_layout()
    plt.show()

    fig.savefig("Images/Pie_chart_Exp.svg", dpi=300, bbox_inches='tight')

    return good_models


In [9]:
def linear_regression(L, A):
    # It is not a linear regression per se but it is used with the curve_fit function to determine 
    # the best linear relation for the parity plot of the estimated data and the tested data.
    return A*np.array(L)

In [10]:
def r_squared_function(y_data, y_fit):
    # Gets the r squared from the fitted data (y_fit) and the reference data (y_data).
    residuals = y_data - y_fit
    ss_res = np.sum(residuals**2)
    
    #You can get the total sum of squares (ss_tot) with
    ss_tot = np.sum((y_data-np.mean(y_data))**2)

    #And finally, the r_squared-value with,
    r_squared = 1 - (ss_res / ss_tot)

    return r_squared